# PyIRoGlass Google Colab

This notebook aims to make running PyIRoGlass possible for all. Please start by running the first cell by clicking the play button on the top left, or shift+enter. This initializes Google Colab, pip installs PyIRoGlass, and downloads some sample spectra and chemistry and thickness spreadsheets from GitHub. 

To load your own spectra, prepare data inputs of your spectra (.CSV) and a spreadsheet called 'ChemThick.CSV' with your chemistry and thickness values. See an example of this file here: https://github.com/sarahshi/PyIRoGlass/blob/main/Inputs/COLAB_BINDER/Colab_Binder_ChemThick.csv. You must make sure that the names of your spectra (everything before the .CSV) must match the names in the Sample column of 'ChemThick.csv'. 

This code allows for FTIR spectra to be batch processed by simply uploading these files onto Google Colab. 

In [ ]:
# %% -*- coding: utf-8 -*-
""" Created on March 24, 2023 // @author: Sarah Shi """

# Import packages

import os
import sys
import glob
import numpy as np
import pandas as pd

!pip install PyIRoGlass 
import PyIRoGlass as pig

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rc, cm

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%cd /content/
!wget -nc https://github.com/sarahshi/PyIRoGlass/blob/main/Inputs/COLAB_BINDER.zip?raw=true
if os.path.exists('COLAB_BINDER')== False:
  !unzip 'COLAB_BINDER.zip?raw=true'

if os.path.exists('YOUR_SPECTRA')== False:
  !mkdir YOUR_SPECTRA
%cd YOUR_SPECTRA/
if os.path.exists('TRANSMISSION')== False:
  !mkdir TRANSMISSION


# Load your spectra

Click the orange folder button on the upper left corner of Google Colab. The files available for you should be visible. Click the refresh button (below 'Files' and the second button from the left with a clockwise arrow) if nothing is showing up. 

You should see the directory 'YOUR_SPECTRA', with a subdirectory 'TRANS' (for transmission FTIR spectra). Click on 'YOUR_SPECTRA', then 'TRANS'. Hover over the right side of 'TRANS' and click the three dots that appear, then click upload. Add all of your spectra CSV to this directory. Now hover over 'YOUR_SPECTRA' and click upload again. Add your file called 'ChemThick.CSV'. Double check that the names of your spectra (everything before the .CSV) match the names in the Sample column of 'ChemThick.csv'. 

Once you upload these files, you're ready to go! 

In [ ]:

# Get working paths 
path_input = os.getcwd() + '/'
path_input

# Change paths to direct to folder with transmission FTIR spectra 
TRANS_PATHS = path_input + 'TRANS/'
TRANS_FILES = sorted(glob.glob(TRANS_PATHS + "*"))

# Put ChemThick file in Inputs. Direct to what your ChemThick file is called. 
CHEMTHICK_PATH = path_input + 'ChemThick.csv'

# Change to be what you want the prefix of your output files to be. 
OUTPUT_PATH = 'RESULTS'


MICOMP, THICKNESS = pig.Load_ChemistryThickness(CHEMTHICK_PATH)

DFS_FILES, DFS_DICT = pig.Load_SampleCSV(TRANS_FILES, wn_high = 5500, wn_low = 1000)
DF_OUTPUT, FAILURES = pig.Run_All_Spectra(DFS_DICT, OUTPUT_PATH)
DF_OUTPUT.to_csv('DF.csv')

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
DENSITY_EPSILON, MEGA_SPREADSHEET = pig.Concentration_Output(DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
MEGA_SPREADSHEET.to_csv('H2OCO2.csv')
DENSITY_EPSILON.to_csv('DensityEpsilon.csv')
MEGA_SPREADSHEET


# Load sample spectra

This implementation in Google Colab also allows for benchmarking against some melt inclusions from Volcán de Fuego and back-arc basin glasses. You can see these in the directory called 'COLAB_BINDER'. The code required for processing these samples is as follows: 

In [ ]:

# Get working paths 
path_input = os.getcwd() + '/COLAB_BINDER/'
path_input

# Change paths to direct to folder with transmission FTIR spectra 
TRANS_PATHS = path_input + 'TRANSMISSION/'
BM_TRANS_FILES = sorted(glob.glob(TRANS_PATHS + "*"))

# Put ChemThick file in Inputs. Direct to what your ChemThick file is called. 
CHEMTHICK_PATH = path_input + 'Colab_Binder_ChemThick.csv'

# Change to be what you want the prefix of your output files to be. 
BM_OUTPUT_PATH = 'BENCHMARK_RESULTS'


BM_MICOMP, BM_THICKNESS = pig.Load_ChemistryThickness(CHEMTHICK_PATH)

BM_DFS_FILES, BM_DFS_DICT = pig.Load_SampleCSV(BM_TRANS_FILES, wn_high = 5500, wn_low = 1000)
BM_DF_OUTPUT, FAILURES = pig.Run_All_Spectra(BM_DFS_DICT, BM_OUTPUT_PATH)
BM_DF_OUTPUT.to_csv('BENCHMARK_DF.csv')

T_ROOM = 25 # C
P_ROOM = 1 # Bar

N = 500000
BM_DENSITY_EPSILON, BM_MEGA_SPREADSHEET = pig.Concentration_Output(BM_DF_OUTPUT, N, THICKNESS, MICOMP, T_ROOM, P_ROOM)
BM_MEGA_SPREADSHEET.to_csv('BENCHMARK_H2OCO2.csv')
BM_DENSITY_EPSILON.to_csv('BENCHMARK_DensityEpsilon.csv')
BM_MEGA_SPREADSHEET


# Thickness Calculation Examples

In [ ]:

# Get working paths 
path_input = os.getcwd() + '/COLAB_BINDER/'
path_input

# Change paths to direct to folder with reflectance FTIR spectra
REF_PATH_OL = path_input + 'REF/OL/'
REF_OL_FILES = sorted(glob.glob(REF_PATH_OL + "*"))

REF_OL_DFS_FILES, REF_OL_DFS_DICT = pig.Load_SampleCSV(REF_OL_FILES, wn_high = 2700, wn_low = 2100)

# Use DHZ parameterization of olivine reflectance index. 
n_ol = pig.Reflectance_Index(0.72)

REF_FUEGO = pig.Thickness_Processing(REF_OL_DFS_DICT, n = n_ol, wn_high = 2700, wn_low = 2100, remove_baseline = True, plotting = False, phaseol = True)

REF_FUEGO


In [ ]:

REF_PATH_GL = path_input + 'REF/GL/'
REF_GL_FILES = sorted(glob.glob(REF_PATH_GL + "*"))

REF_GL_DFS_FILES, REF_GL_DFS_DICT = pig.Load_SampleCSV(REF_GL_FILES, wn_high = 2850, wn_low = 1700)

# n=1.546 in the range of 2000-2700 cm^-1 following Nichols and Wysoczanski, 2007 for basaltic glass
n_gl = 1.546

REF_GLASS = pig.Thickness_Processing(REF_GL_DFS_DICT, n = n_gl, wn_high = 2850, wn_low = 1700, remove_baseline = True, plotting = False, phaseol = False)

REF_GLASS
